# Neural Network

## Read data

In [ ]:
import pandas as pd

df_train = pd.read_json("../data/aclIMDB_train.json", orient="records")
df_train.info()
df_train.hist()
df_train.head()

In [ ]:
df_val = pd.read_json("../data/aclIMDB_val.json", orient="records")
df_val.info()
df_val.hist()
df_val.head()

In [ ]:
df_test = pd.read_json("../data/aclIMDB_test.json", orient="records")
df_test.info()
df_test.hist()
df_test.head()

## Preprocessing

In [ ]:
# !pip install nltk

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

STOPWORDS = stopwords.words("english")

def is_html_tag(word):
    w = word.replace("\n", "")
    return w.startswith("<") or w.startswith(">") or w.startswith("/") or w.strip()[:2] == "br"

def remove_html_tags(sentence):
    single_spaces = " ".join(sentence.split())

    return " ".join([token for token in single_spaces.split(" ") if not is_html_tag(token)])

def tokenize_sentence(sentence):
    return word_tokenize(sentence.lower())

def remove_stopwords(sentence):
    return " ".join([word for word in sentence.split(" ") if not word in STOPWORDS])

def remove_punctuation(tokenized_sentence):
    return [word for word in tokenized_sentence if word.isalpha()]

def preprocess(sentence):
    return remove_punctuation(tokenize_sentence(remove_stopwords(remove_html_tags(str(sentence)))))

print(preprocess(df_train["text"].values[1]))


In [ ]:
def sentiment(value):
    if(value == 1):
        return "positive"
    else:
        return "negative"

df_train["sentiment"] = [None] * len(df_train)
df_train["sentiment"] = df_train["label"].apply(sentiment)
df_val["sentiment"] = [None] * len(df_test)
df_val["sentiment"] = df_val["label"].apply(sentiment)
df_test["sentiment"] = [None] * len(df_test)
df_test["sentiment"] = df_test["label"].apply(sentiment)

In [ ]:
df_train["text"] = df_train["text"].apply(preprocess)
df_train.head()

In [ ]:
def build_vocab(tokenized_input, vocab_size):
    d = dict()

    for tokens in tokenized_input:
        for token in tokens:
            # double check
            if token not in STOPWORDS and token.isalpha():
                d[token] = d.get(token, 0) + 1

    del d["br"]

    return {k for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)[:vocab_size]}

In [ ]:
VOCAB_SIZE = 10000
VOCAB = build_vocab(df_train["text"].values, VOCAB_SIZE) # (!) Always build the vocabulary on the TRAIN dataset

In [ ]:
def get_frequencies_for_sentiment(df):
    dict_freqs = {"positive": {}, "negative": {}}
    
    for idx in range(df.shape[0]):
        tokens = df_train.iloc[idx]["text"]
        sentiment = df_train.iloc[idx]["sentiment"]
        
        for token in tokens:
            if token in VOCAB:
                dict_freqs[sentiment][token] = dict_freqs[sentiment].get(token, 0) + 1
            
    return dict_freqs

In [ ]:
frequency_table = get_frequencies_for_sentiment(df_train)
frequency_table["positive"]["happy"]

In [ ]:
import numpy as np

def extract_features(frequency_table, tokens):
    positives = 0
    negatives = 0

    for t in tokens:
        positives += frequency_table["positive"].get(t, 0)
        negatives += frequency_table["negative"].get(t, 0)
    
    return pd.Series({"positives": positives, "negatives": negatives})

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = df_train["text"].apply(lambda tokens: extract_features(frequency_table, tokens))

X_train = scaler.fit_transform(X_train)
y_train = df_train["label"].values

In [ ]:
X_train.shape

In [ ]:
df_val["text"] = df_val["text"].apply(preprocess)
X_val = df_val["text"].apply(lambda tokens: extract_features(frequency_table, tokens))
X_val = scaler.transform(X_val)
y_val = df_val["label"].values

## Create model and train / validate

In [ ]:
# !pip install tensorflow

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(2,)),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# https://keras.io/api/optimizers/
# https://keras.io/api/optimizers/adam/
# https://keras.io/api/losses/
# https://keras.io/api/losses/probabilistic_losses/#binarycrossentropy-class
# https://keras.io/api/metrics/accuracy_metrics/
# https://keras.io/api/metrics/
model.compile(optimizer='adam',
              loss="binary_crossentropy",
              metrics=['binary_accuracy'])
              
model.summary()

In [ ]:
h = model.fit(X_train,
              y_train,
              validation_data=(X_val, y_val),
              epochs=10)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(h.history['loss'], label='(training data)',color='blue')
plt.plot(h.history['val_loss'], label='(validation data)',color='green')
plt.title('Neural Network training loss')
plt.ylabel('value')
plt.xlabel('No. epoch')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(h.history['binary_accuracy'], label='(training data)',color='blue')
plt.plot(h.history['val_binary_accuracy'], label='(validation data)',color='green')
plt.title('Neural Network training accuracy')
plt.ylabel('value')
plt.xlabel('No. epoch')
plt.show()

## Evaluate the trained model

In [ ]:
df_test["text"] = df_test["text"].apply(preprocess)
X_test = df_test["text"].apply(lambda tokens: extract_features(frequency_table, tokens))
X_test = scaler.transform(X_test)
y_test = df_test["label"].values

In [ ]:
results = model.evaluate(X_test, y_test, batch_size=50)
print("test loss, test acc:", results)

In [ ]:
df_test_raw = pd.read_json("../data/aclIMDB_test.json", orient="records")

In [ ]:
predictions = np.round(model.predict(X_test[:5]),0)
for i in range(len(predictions)):
    print(df_test_raw["text"].values[i][:100],"...")
    print("Pred: ", predictions[i][0], "Real: ", y_test[i])